# Tệp nhật ký - Khám phá dữ liệu

## Thư viện được sử dụng

In [1]:
from pyspark.sql import SparkSession

## Tạo đối tượng cho Spark
Avoid org.apache.spark.SparkUpgradeException: Có thể nhận được kết quả khác do nâng cấp Spark 3.0

In [2]:
spark = SparkSession\
.builder\
.appName("pyspark-notebook").\
config("spark.sql.legacy.timeParserPolicy", "LEGACY").\
getOrCreate()

24/12/01 06:53:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Tải dữ liệu đầu vào

In [3]:
log_file_path = r"actual_log.txt"

In [4]:
base_df = spark.read.text(log_file_path)
# Let's look at the schema
base_df.printSchema()
base_df.show(truncate=False)

root
 |-- value: string (nullable = true)



+-------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                          |
+-------------------------------------------------------------------------------------------------------------------------------+
|in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839     |
|uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304                                                          |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0                               |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304                               |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0"

##  Phân tích

Nếu đã quen thuộc với máy chủ web, chúng ta sẽ nhận ra rằng đây là <a href="https://www.w3.org/Daemon/User/Config/Logging.html#common-logfile-format" > Định dạng nhật ký chung </a>. Các trường là:
<b>remotehost rfc931 authuser date "request" status bytes</b>

<table>
    <tr>
        <th>Trường</th>
        <th>Ý nghĩa</th>
    </tr>
    <tr>
        <td>remotehost</td>
        <td>Tên máy chủ từ xa (hoặc số IP nếu tên máy chủ DNS không có sẵn).</td>
    </tr>
    <tr>
        <td>rfc931</td>
        <td>Tên đăng nhập từ xa của người dùng.</td>
    </tr>
    <tr>
        <td>authuser</td>
        <td>Tên người dùng của người dùng từ xa, được xác thực bởi máy chủ HTTP.</td>
    </tr>
    <tr>
        <td>date</td>
        <td>Ngày và giờ của yêu cầu.</td>
    </tr>
    <tr>
        <td>request</td>
        <td>Yêu cầu chính xác như được gửi từ trình duyệt hoặc ứng dụng khách.</td>
    </tr>
    <tr>
        <td>status</td>
        <td>Mã trạng thái HTTP mà máy chủ gửi lại cho máy khách.</td>
    </tr>
    <tr>
        <td>bytes</td>
        <td>Số byte (Content-Length) được truyền tới máy khách.</td>
    </tr>
</table>
Tiếp theo, chúng ta phải phân tích nó thành các cột riêng lẻ. Chúng ta sẽ sử dụng hàm regrec_extract() đặc biệt được tích hợp sẵn để thực hiện phân tích cú pháp. Hàm này khớp một cột với biểu thức chính quy với một hoặc nhiều nhóm chụp và cho phép bạn trích xuất một trong các nhóm phù hợp. Chúng tôi sẽ sử dụng một biểu thức chính quy cho mỗi trường mà chúng tôi muốn trích xuất.
<br> </br>   
Nếu cảm thấy biểu thức chính quy khó hiểu và muốn khám phá thêm về chúng, hãy bắt đầu với <a href="https://regexone.com/">RegexOne web site</a>.

In [5]:
from pyspark.sql.functions import split, regexp_extract
split_df = base_df.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('host'),
                          regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('timestamp'),
                          regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('path'),
                          regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('status'),
                          regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('content_size'))
split_df.show(truncate=False)

+----------------------------+--------------------------+---------------------------------------------------+------+------------+
|host                        |timestamp                 |path                                               |status|content_size|
+----------------------------+--------------------------+---------------------------------------------------+------+------------+
|in24.inetnebr.com           |01/Aug/1995:00:00:01 -0400|/shuttle/missions/sts-68/news/sts-68-mcc-05.txt    |200   |1839        |
|uplherc.upl.com             |01/Aug/1995:00:00:07 -0400|/                                                  |304   |null        |
|uplherc.upl.com             |01/Aug/1995:00:00:08 -0400|/images/ksclogo-medium.gif                         |304   |0           |
|uplherc.upl.com             |01/Aug/1995:00:00:08 -0400|/images/MOSAIC-logosmall.gif                       |304   |null        |
|uplherc.upl.com             |01/Aug/1995:00:00:08 -0400|/images/USA-logosmall.gif        

## Cleaning
Hãy xem logic phân tích cú pháp của chúng tôi hoạt động tốt như thế nào. Trước tiên, hãy xác minh rằng không có hàng rỗng trong tập dữ liệu gốc.

In [6]:
base_df.filter(base_df['value'].isNull()).count()

0

Điều đó có nghĩa là không có dòng nào có giá trị null. Hãy kiểm tra khung dữ liệu được phân tích cú pháp

In [7]:
bad_rows_df = split_df.filter(split_df['host'].isNull() |
                              split_df['timestamp'].isNull() |
                              split_df['path'].isNull() |
                              split_df['status'].isNull() |
                             split_df['content_size'].isNull())
bad_rows_df.count()

2

Một số trường không được điền vào dữ liệu gốc. Chúng ta cần tìm ra lĩnh vực nào bị ảnh hưởng.

In [8]:
from pyspark.sql.functions import col, sum

def count_null(col_name):
  return sum(col(col_name).isNull().cast('integer')).alias(col_name)
exprs = []
[exprs.append(count_null(col_name)) for col_name in split_df.columns]
split_df.agg(*exprs).show()

+----+---------+----+------+------------+
|host|timestamp|path|status|content_size|
+----+---------+----+------+------------+
|   0|        0|   0|     0|           2|
+----+---------+----+------+------------+



## Sửa các hàng có nội dung null_size

Giải pháp đơn giản nhất là thay thế các giá trị null trong Split_df bằng 0. API DataFrame cung cấp một tập hợp các hàm và trường được thiết kế riêng để làm việc với các giá trị null. Chúng ta sử dụng na để thay thế <code>content_size</code> bằng 0.

In [9]:
cleaned_df = split_df.na.fill({'content_size': 0})
exprs = []
[exprs.append(count_null(col_name)) for col_name in cleaned_df.columns]

cleaned_df.agg(*exprs).show()

+----+---------+----+------+------------+
|host|timestamp|path|status|content_size|
+----+---------+----+------+------------+
|   0|        0|   0|     0|           0|
+----+---------+----+------+------------+



In [10]:
from pyspark.sql.functions import *
logs_df = cleaned_df.select('*', to_timestamp(cleaned_df['timestamp'],"dd/MMM/yyyy:HH:mm:ss ZZZZ").cast('timestamp').alias('time')).drop('timestamp')
total_log_entries = logs_df.count()
print(total_log_entries)
logs_df.show(truncate=False)

20
+----------------------------+---------------------------------------------------+------+------------+-------------------+
|host                        |path                                               |status|content_size|time               |
+----------------------------+---------------------------------------------------+------+------------+-------------------+
|in24.inetnebr.com           |/shuttle/missions/sts-68/news/sts-68-mcc-05.txt    |200   |1839        |1995-08-01 04:00:01|
|uplherc.upl.com             |/                                                  |304   |0           |1995-08-01 04:00:07|
|uplherc.upl.com             |/images/ksclogo-medium.gif                         |304   |0           |1995-08-01 04:00:08|
|uplherc.upl.com             |/images/MOSAIC-logosmall.gif                       |304   |0           |1995-08-01 04:00:08|
|uplherc.upl.com             |/images/USA-logosmall.gif                          |304   |0           |1995-08-01 04:00:08|
|ix-esc-ca2-0

In [11]:
#01/Aug/1995:00:00:07 -0400
cleaned_df.select('*').withColumn('time',to_timestamp(col("timestamp"),'dd/MMM/yyyy:HH:mm:ss ZZZZ')) \
  .show(truncate=False)

+----------------------------+--------------------------+---------------------------------------------------+------+------------+-------------------+
|host                        |timestamp                 |path                                               |status|content_size|time               |
+----------------------------+--------------------------+---------------------------------------------------+------+------------+-------------------+
|in24.inetnebr.com           |01/Aug/1995:00:00:01 -0400|/shuttle/missions/sts-68/news/sts-68-mcc-05.txt    |200   |1839        |1995-08-01 04:00:01|
|uplherc.upl.com             |01/Aug/1995:00:00:07 -0400|/                                                  |304   |0           |1995-08-01 04:00:07|
|uplherc.upl.com             |01/Aug/1995:00:00:08 -0400|/images/ksclogo-medium.gif                         |304   |0           |1995-08-01 04:00:08|
|uplherc.upl.com             |01/Aug/1995:00:00:08 -0400|/images/MOSAIC-logosmall.gif               